# 1. Notebook Setup
# Title, assignment info, and markdown overview.

In [81]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random as rd
import csv
from typing import Union

In [82]:
mapping_file_path = "data/Mappings.csv"
employees_file_path = "data/Employee.csv"
tasks_file_path = "data/Tasks.csv"
rd.seed(20)

# Data Classes
class Task:
    time:int =0
    difficulty:int =0
    deadline:int = 0
    required_skill:str = ''
    def __str__(self):
        return f"T Time:{self.time} Difficulty:{self.difficulty} Deadline:{self.deadline} requiredSkill:{self.required_skill}"

class Employee:
    available_hours:int = 0
    skill_level:int = 4
    skills:list[str]=['']

    def __str__(self):
        return f"E AH:{self.available_hours} Skill-Level:{self.skill_level} Skills:{self.skills}"




In [86]:
# I/O
"""
Task csv:

'ID', 'Time (hrs)', 'Difficulty', 'Deadline (hrs)', 'Required Skill'

Employee csv:
'Employee ID', 'Available Hrs', 'Skill Level', 'Skills'


Using an adjancey list instead of an adjacney matrix!

Since the input vector is 10 mappings x 11 features. Including a unique assignment penalty is redundant since a task mapped to 2 employees will create (10+1) mappings which doesn't work for the network input layer!
"""




class DataLoader:
    num_of_tasks =10
    num_of_employees = 5
    tasks:list[Task]=[]
    employees:list[Employee]=[]
    
    def load_tasks(self,fileName=tasks_file_path):
        tasks =[]
        with open(fileName,'r') as csvfile:
            taskReader = csv.reader(csvfile)
            next(taskReader)
            for taskArr in taskReader:
                newT = Task()
                newT.time=int(taskArr[1])
                newT.difficulty= int(taskArr[2])
                newT.deadline = int(taskArr[3])
                newT.required_skill= taskArr[4]
                tasks.append(newT)
        self.tasks= tasks

    def load_employees(self,fileName=employees_file_path):
        employees=[]
        with open(fileName,'r') as csvfile:
            employeeReader = csv.reader(csvfile)
            next(employeeReader)
            for employeeArr in employeeReader:
                newEmployee = Employee()
                newEmployee.available_hours=int(employeeArr[1])
                newEmployee.skill_level= int(employeeArr[2])
                skills = employeeArr[3].split(',')
                newEmployee.skills = skills
                employees.append(newEmployee)
        self.employees = employees
    def loadAll(self):
        self.load_employees()
        self.load_tasks()
    


class MappingHandler:
    def __init__(self,tasks:list[Task],employees:list[Employee],num_of_mappings=100):
        self.tasks:list[Task]= tasks
        self.employees:list[Employee] = employees
        self.num_of_mappings = num_of_mappings
        self.mappings = []
        self.costs=[]


    def __costFunction(self,Mapping:list[int]):
        w=0.2
        overload = 0
        skill_mismatch = 0
        difficulty_violation = 0
        deadline_violation = 0
        unique_assignment = 0
        employee_task_adj_list:list[list[Task]]=[[] for _ in range(len(self.employees))]

        for i in range(len(Mapping)):
            taskId=i

            task = self.tasks[taskId]
            num_of_employees_assigned = 0
            employeeId = Mapping[i]
            num_of_employees_assigned+=1
            employee = self.employees[employeeId]
            employee_task_adj_list[employeeId].append(task)
            # skill mismatch violation
            if task.required_skill not in employee.skills:
                skill_mismatch+=1
            if task.difficulty> employee.skill_level:
                difficulty_violation+= 1
            unique_assignment+= max(0,num_of_employees_assigned-1)



        for employeeId in range(len(employee_task_adj_list)):
            employee = self.employees[employeeId]
            sortedEmployeeTasks = employee_task_adj_list[employeeId]
            sortedEmployeeTasks.sort(key= lambda x:x.time)
            sumHours=0
            finishTime=0
            for t in sortedEmployeeTasks:
                sumHours+=t.time
                finishTime+= t.time
                deadline_violation+= max(0,finishTime - t.deadline)
            overload+= max(0,sumHours- employee.available_hours) 
        total_penalty = overload+skill_mismatch+unique_assignment+deadline_violation+ difficulty_violation
        return round(total_penalty*w,3)
    def __reset(self):
        self.mappings=[]
        self.costs=[]


    def generateMappings(self):
        unique = set()
        self.__reset()
        while len(unique) < self.num_of_mappings:
            mapping:list[int] = [0 for _ in self.tasks]         # Create a mapping for each iteration. Using an adjlist format
            # possible_assignments = [i for i in range(len(employees))]  #list of possible assignments
            for taskId in range(len(self.tasks)):
                rand_employee =rd.randint(0,len(self.employees)-1)
                mapping[taskId]=rand_employee
            string = str(mapping)
            if string not in unique: # ensure no duplicates in data generation
                unique.add(string)
                self.mappings.append(mapping)
                self.costs.append(self.__costFunction(mapping))
    

    def readCSV(self,fileName=mapping_file_path):
        self.__reset()
        with open(fileName,'r') as csvfile:
            mappingReader = csv.reader(csvfile)
            next(mappingReader)
            for line in mappingReader:
                newMapping:list[int]=[]
                for v in line:
                    if v.isdigit():
                        newMapping.append(int(v))
                    else:
                        self.costs.append(float(v))

                self.mappings.append(newMapping)


    def writeCSV(self,filepath =mapping_file_path):
        with open(filepath,'w',newline='') as csvfile:
                mappingWriter = csv.writer(csvfile)
                mappingWriter.writerow(["T1",'T2','T3','T4','T5','T6','T7','T8','T9','T10','Penalty'])
                mappingWriter.writerows(self.mappings)
        





def testDataGen():
    dl = DataLoader()
    dl.loadAll()
    mappingloader = MappingHandler(dl.tasks,dl.employees)
    mappingloader.generateMappings()
    mappingloader.writeCSV('data/Mappings.csv')


testDataGen()


# def TestDataGen():

#     all_mappings = mappingGenerator(task,employees)
#     for mapping in all_mappings:
#         cost = costFunction(mapping,task,employees)
#         print(cost)

# def verifyCost():
#     mapping=[[2],[3],[1],[4],[2],[5],[1],[3],[5],[4]]
    
#     for i in range(len(mapping)):
#         mapping[i][0]-=1
#     employees,tasks = DataLoader().loadAll()
#     # for t in tasks:
#     #     print(t)
    
#     for e in employees:
#         print(e)
#     print(costFunction(mapping,tasks,employees))





In [ ]:
# 4. Preprocessing
# returns a 3-element vector
def one_hot_encode_skill(skillsVector):
    skills= ['A','B','C']
    hot_encoding=[]
    j=0
    for i in range(len(skills)):
        if j< len(skillsVector)and  skills[i] == skillsVector[j]:
            j+=1
            hot_encoding.append(1)
        else:
            hot_encoding.append(0)
    return hot_encoding





def construct_input_vector(mappingloader:MappingHandler):
    featureVector = []

    for mp in mappingloader.mappings:
        vector=[]
        for i in range(len(mp)):

            task = mappingloader.tasks[i]
            employee = mappingloader.employees[mp[i]]
            vector.extend([task.time,task.difficulty,task.deadline])
            vector.extend(one_hot_encode_skill(task.required_skill))
            vector.extend([employee.available_hours,employee.skill_level])
            vector.extend(one_hot_encode_skill( employee.skills))
        featureVector.append(vector)
        
    df = pd.DataFrame(featureVector)
    print(df.shape)
    return df

def split_data(df):
    df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)
    train_size = int(0.7 * len(df_shuffled))
    val_size = int(0.15 * len(df_shuffled))
    test_size = len(df_shuffled) - train_size - val_size
    train_df = df_shuffled[:train_size]
    val_df = df_shuffled[train_size:train_size + val_size]
    test_df = df_shuffled[train_size + val_size:]
    return train_df,val_df,test_df

def pre_process():
    dl = DataLoader()
    dl.loadAll()
    mappingloader = MappingHandler(dl.tasks,dl.employees)
    mappingloader.readCSV('data/Mappings.csv')
    df = construct_input_vector(mappingloader)
    return split_data(df)

# pre_process()




(100, 110)
    0    1    2    3    4    5    6    7    8    9    ...  100  101  102  103  \
0     4    3    8    1    0    0    9    5    1    0  ...    4   11    0    0   
1     4    3    8    1    0    0    8    3    1    0  ...    4   11    0    0   
2     4    3    8    1    0    0    9    5    1    0  ...    4   11    0    0   
3     4    3    8    1    0    0   10    4    1    0  ...    4   11    0    0   
4     4    3    8    1    0    0   15    7    0    1  ...    4   11    0    0   
..  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
65    4    3    8    1    0    0   10    4    1    0  ...    4   11    0    0   
66    4    3    8    1    0    0   12    6    1    1  ...    4   11    0    0   
67    4    3    8    1    0    0    8    3    1    0  ...    4   11    0    0   
68    4    3    8    1    0    0   15    7    0    1  ...    4   11    0    0   
69    4    3    8    1    0    0    9    5    1    0  ...    4   11    0    0   

    104  105  10

(    0    1    2    3    4    5    6    7    8    9    ...  100  101  102  103  \
 0     4    3    8    1    0    0    9    5    1    0  ...    4   11    0    0   
 1     4    3    8    1    0    0    8    3    1    0  ...    4   11    0    0   
 2     4    3    8    1    0    0    9    5    1    0  ...    4   11    0    0   
 3     4    3    8    1    0    0   10    4    1    0  ...    4   11    0    0   
 4     4    3    8    1    0    0   15    7    0    1  ...    4   11    0    0   
 ..  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
 65    4    3    8    1    0    0   10    4    1    0  ...    4   11    0    0   
 66    4    3    8    1    0    0   12    6    1    1  ...    4   11    0    0   
 67    4    3    8    1    0    0    8    3    1    0  ...    4   11    0    0   
 68    4    3    8    1    0    0   15    7    0    1  ...    4   11    0    0   
 69    4    3    8    1    0    0    9    5    1    0  ...    4   11    0    0   
 
     104  105 

In [ ]:
# 5. Model Definitions




class NeuralNetwrokArgs:
    layer_dims=[]
    name=''
    activation=''

class NeuralNetwork:
    def __init__(self, layer_dims:list[int],name, activation='relu'):
        self.name = name
        self.input_size = layer_dims[0]
        self.weights=np.array([])
        self.biases=np.array([])
        
        for i in range(1,len(layer_dims)):
            self.weights= np.append(self.weights,np.random.randn(layer_dims[i],layer_dims[i-1]))*0.01
            self.biases =np.append(self.biases,np.zeros((layer_dims[i],1)))
        

        self.layer_dims = layer_dims
    def forward(self, x):
        

    def backward(self, x, y_true):
        pass

    def update_params(self, lr):
        pass

    # 6. Training Loop
    def train(model, X_train, y_train, params):
        pass
    # implement mini-batch SGD, record loss

# 7. Evaluation & Plots
#- Generate the eight required figures
#- Save each via plt.savefig()
# 8. Save & Export
#- Download figures
#- Optionally, pickle model parameters

SyntaxError: invalid character '’' (U+2019) (3784911823.py, line 3)